In [6]:
%pip install scholarly

^C
Note: you may need to restart the kernel to use updated packages.


     ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
     --------------------------------------- 0.1/10.0 MB 409.6 kB/s eta 0:00:25
     --------------------------------------- 0.1/10.0 MB 654.9 kB/s eta 0:00:16
      -------------------------------------- 0.2/10.0 MB 841.6 kB/s eta 0:00:12
     - -------------------------------------- 0.3/10.0 MB 1.0 MB/s eta 0:00:10
     - -------------------------------------- 0.4/10.0 MB 1.4 MB/s eta 0:00:07
     -- ------------------------------------- 0.6/10.0 MB 1.7 MB/s eta 0:00:06
     -- ------------------------------------- 0.6/10.0 MB 1.7 MB/s eta 0:00:06
     --- ------------------------------------ 0.8/10.0 MB 1.8 MB/s eta 0:00:06
     ---- ----------------------------------- 1.2/10.0 MB 2.3 MB/s eta 0:00:04
     ------ --------------------------------- 1.6/10.0 MB 3.0 MB/s eta

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from scholarly import scholarly

In [2]:
# Read the CSV file
df = pd.read_csv('csrankings.csv')

# Define keywords for each field
system_keywords = ['system', 'architecture', 'hardware']
ai_keywords = ['artificial intelligence', 'machine learning', 'deep learning', 'computer vision']
theory_keywords = ['theory', 'algorithm', 'computational complexity']
interdisciplinary_keywords = ['interdisciplinary', 'cross-disciplinary', 'multi-disciplinary']

In [3]:
# Create columns for each field
df['System'] = False
df['AI'] = False
df['Theory'] = False
df['Interdisciplinary'] = False
df['Publications'] = 0
df['Citations'] = 0


In [4]:
# Function to check keywords in the webpage content
def check_keywords(url, keywords):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        interests = soup.find('div', class_='interests').text  # Replace with the actual HTML structure
        return any(keyword in interests.lower() for keyword in keywords)
    except Exception as e:
        print(f"Error accessing {url}: {e}")
        return False

In [5]:
print(df)

                     name                      affiliation  \
0              A Min Tjoa                          TU Wien   
1       A. Akbari Azirani                             IUST   
2        A. Akbariazirani                             IUST   
3          A. Aldo Faisal          Imperial College London   
4      A. Antony Franklin                    IIT Hyderabad   
...                   ...                              ...   
28481        Özgür Erkent             Hacettepe University   
28482        Özgür Ulusoy               Bilkent University   
28483        Öznur Tastan               Sabancı University   
28484       Öznur Özkasap                   Koç University   
28485  Ümit V. Çatalyürek  Georgia Institute of Technology   

                                                homepage     scholarid  \
0                       http://www.ifs.tuwien.ac.at/tjoa  x8qCMhcAAAAJ   
1      http://ce.iust.ac.ir/page.php?slct_pg_id=6537&...  pCil4_cAAAAJ   
2      http://ce.iust.ac.ir/page.

In [6]:
def search_google_scholar(author_id, author_name):
    try:
        if author_id != 'NOSCHOLARPAGE':
            author = scholarly.search_author_id(author_id)
        else:
            search_query = scholarly.search_author(author_name)
            author = next(search_query)
        return author
    except Exception as e:
        print(f"Exception for author with ID {author_id}: {e}")
        return None

In [7]:
for index, row in df.iterrows():
    author_id = str(row['scholarid'])  # Replace 'GoogleScholarID' with the actual column name
    author_name = str(row['name'])
    author_info = search_google_scholar(author_id, author_name)

    if author_info:
        df.at[index, 'Citations'] = author_info.get('citedby', 0)

        interests = author_info.get('interests', [])
        df.at[index, 'System'] = any(keyword in interests for keyword in system_keywords)
        df.at[index, 'AI'] = any(keyword in interests for keyword in ai_keywords)
        df.at[index, 'Theory'] = any(keyword in interests for keyword in theory_keywords)
        df.at[index, 'Interdisciplinary'] = any(keyword in interests for keyword in interdisciplinary_keywords)

# Save the updated DataFrame to a new CSV file
df.to_csv('tagged_faculty_data.csv', index=False)

Exception for author with ID LVfqLuoAAAAJ: Cannot Fetch from Google Scholar.
Exception for author with ID n25sJzkAAAAJ: Cannot Fetch from Google Scholar.


KeyboardInterrupt: 

In [ ]:
print(author_info)

{'container_type': 'Author', 'filled': ['basics'], 'scholar_id': 'mF-F-zkAAAAJ', 'source': <AuthorSource.AUTHOR_PROFILE_PAGE: 'AUTHOR_PROFILE_PAGE'>, 'name': 'Anand Madhu Kumar', 'url_picture': 'https://scholar.googleusercontent.com/citations?view_op=view_photo&user=mF-F-zkAAAAJ&citpid=2', 'affiliation': 'Indie', 'interests': ['Neuroscience', 'Egyptology', 'Economics', 'Humanities'], 'email_domain': '@amonereb.com', 'homepage': 'https://www.instagram.com/drewwgie/'}


In [ ]:
author_info['filled']

['basics']

In [24]:
# Retrieve the author's data, fill-in, and print
print(scholarly.search_author_id('vY7MdLYAAAAJ'))

{'container_type': 'Author', 'filled': ['basics'], 'scholar_id': 'vY7MdLYAAAAJ', 'source': <AuthorSource.AUTHOR_PROFILE_PAGE: 'AUTHOR_PROFILE_PAGE'>, 'name': 'Kang G. Shin', 'url_picture': 'https://scholar.googleusercontent.com/citations?view_op=view_photo&user=vY7MdLYAAAAJ&citpid=4', 'affiliation': "Kevin & Nancy O'Connor Professor of Computer Science, University of Michigan", 'organization': 4770128543809686866, 'interests': ['Real-time embedded systems', 'cyber-physical systems', 'computer networks', 'security and privacy', 'mobile and wireless computing and c'], 'email_domain': '@umich.edu', 'homepage': 'http://www.eecs.umich.edu/~kgshin', 'citedby': 59065}


In [ ]:
search_query = scholarly.search_author("A. Aldo Faisal")

In [8]:
author_info = search_google_scholar("NOSCHOLARPAGE", 'A. T. Chamillard')

if author_info:
    df.at[index, 'Citations'] = author_info.get('citedby', 0)

    interests = author_info.get('interests', [])
    df.at[index, 'System'] = any(keyword in interests for keyword in system_keywords)
    df.at[index, 'AI'] = any(keyword in interests for keyword in ai_keywords)
    df.at[index, 'Theory'] = any(keyword in interests for keyword in theory_keywords)
    df.at[index, 'Interdisciplinary'] = any(keyword in interests for keyword in interdisciplinary_keywords)

Exception for author with ID NOSCHOLARPAGE: 


In [ ]:
mDLQC0EAAAAJ

In [19]:
search_query = scholarly.search_author('A. T. Chamillard')

In [21]:
if not any(search_query):
    print("Iterable is empty")
else:
    for sq in search_query:
        print(sq)


Iterable is empty


In [16]:
if not search_query:
    print("Iterable is empty")
else:
    print("Iterable is not empty")

Iterable is not empty


In [10]:
print(author)

None
